# This is for us to create a ML model to predict future housing prices 


In [70]:
# Load dataset
df = pd.read_csv("/Users/belinda/Desktop/DAC-Internal-Projects/dataset/updated_dataset.csv")


In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn import tree
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [72]:
#  Adding a row for date time on the remaining lease
# Load the dataset
df = pd.read_csv("/Users/belinda/Desktop/DAC-Internal-Projects/dataset/updated_dataset.csv")

# Convert 'month' column to datetime format
df['month'] = pd.to_datetime(df['month'], errors='coerce')

# Function to calculate lease expiry date
def calculate_lease_expiry(row):
    try:
        # Parse the remaining lease (e.g., "61 years 04 months")
        duration_parts = row['remaining_lease'].split(' ')
        years = int(duration_parts[0])  # Extract years
        months = int(duration_parts[2])  # Extract months
        # Add the duration to the 'month' column
        return row['month'] + relativedelta(years=years, months=months)
    except Exception as e:
        return None  # Return None if parsing fails

# Apply the function to calculate the new lease expiry dates
df['remaining_lease'] = df.apply(calculate_lease_expiry, axis=1)

print(df.head())


       month        town flat_type block        street_name storey_range  \
0 2017-01-01  ANG MO KIO    2 ROOM   406  ANG MO KIO AVE 10     10 TO 12   
1 2017-01-01  ANG MO KIO    3 ROOM   108   ANG MO KIO AVE 4     01 TO 03   
2 2017-01-01  ANG MO KIO    3 ROOM   602   ANG MO KIO AVE 5     01 TO 03   
3 2017-01-01  ANG MO KIO    3 ROOM   465  ANG MO KIO AVE 10     04 TO 06   
4 2017-01-01  ANG MO KIO    3 ROOM   601   ANG MO KIO AVE 5     01 TO 03   

   floor_area_sqm      flat_model lease_commence_date remaining_lease  \
0            44.0        Improved          1979-01-01      2078-05-01   
1            67.0  New Generation          1978-01-01      2077-08-01   
2            67.0  New Generation          1980-01-01      2079-06-01   
3            68.0  New Generation          1980-01-01      2079-02-01   
4            67.0  New Generation          1980-01-01      2079-06-01   

   resale_price  
0      232000.0  
1      250000.0  
2      262000.0  
3      265000.0  
4      265000.

In [73]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979-01-01,2078-05-01,232000.0
1,2017-01-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978-01-01,2077-08-01,250000.0
2,2017-01-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980-01-01,2079-06-01,262000.0
3,2017-01-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980-01-01,2079-02-01,265000.0
4,2017-01-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980-01-01,2079-06-01,265000.0


In [74]:
# Pre-processing


df = df.dropna()

# Feature engineering: Calculate remaining lease in years
current_year = pd.Timestamp.now().year
df['remaining_lease_years'] = df['lease_commence_date'].apply(lambda x: (pd.Timestamp(x).year - current_year))





#Feature engineering: Create an 'affordable' column (binary target)
median_income = 5000  # Example median income
affordability_threshold = median_income * 0.3 * 12  # Annual affordability threshold
df['affordable'] = (df['resale_price'] <= affordability_threshold).astype(int)



# Normalize numerical columns
scaler = StandardScaler()
df[['floor_area_sqm', 'remaining_lease_years', 'resale_price']] = scaler.fit_transform(
    df[['floor_area_sqm', 'remaining_lease_years', 'resale_price']]
)

# Encode categorical variables
encoder = LabelEncoder()
df['town'] = encoder.fit_transform(df['town'])
df['flat_type'] = encoder.fit_transform(df['flat_type'])

# Split data into features and target
X = df.drop(['affordable', 'resale_price', 'remaining_lease'], axis=1)  # Features
y = df['affordable']  # Target

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verify the shapes of the split data
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (142334, 10)
X_test shape: (35584, 10)
y_train shape: (142334,)
y_test shape: (35584,)


In [75]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


TypeError: float() argument must be a string or a number, not 'Timestamp'

In [65]:
import matplotlib.pyplot as plt

# Feature importance
importances = model.feature_importances_
feature_names = X.columns
sorted_indices = importances.argsort()

plt.figure(figsize=(10, 6))
plt.barh(range(len(importances)), importances[sorted_indices], align='center')
plt.yticks(range(len(importances)), feature_names[sorted_indices])
plt.xlabel("Feature Importance")
plt.title("Random Forest Feature Importance")
plt.show()


AttributeError: 'RandomForestClassifier' object has no attribute 'estimators_'

In [ ]:
import seaborn as sns

# Aggregate affordability by town
df['town_name'] = encoder.inverse_transform(df['town'])  # Convert encoded towns back
affordability_summary = df.groupby('town_name')['affordable'].mean().reset_index()
affordability_summary.columns = ['Town', 'Affordability_Percentage']

# Plot heatmap
plt.figure(figsize=(12, 6))
sns.barplot(data=affordability_summary, x='Town', y='Affordability_Percentage', palette='viridis')
plt.xticks(rotation=45)
plt.title("Affordability Percentage by Town")
plt.ylabel("Percentage of Affordable Flats")
plt.show()


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# Train a Gradient Boosting Regressor
regressor = GradientBoostingRegressor(random_state=42)
regressor.fit(X_train, df.loc[X_train.index, 'resale_price'])

# Predict resale prices
y_pred_price = regressor.predict(X_test)

# Evaluate model
rmse = mean_squared_error(df.loc[X_test.index, 'resale_price'], y_pred_price, squared=False)
print("Root Mean Squared Error:", rmse)

# Calculate affordability score
affordability_scores = df.loc[X_test.index, 'resale_price'] - y_pred_price
df.loc[X_test.index, 'affordability_score'] = affordability_scores
